Guillaume Tochon <br>
TP OCVX2 <br>
Promo IMAGE 2025 <br>

# Prise en main des solveurs d'optimisation

Avant de plonger dans la résolution des SVMs, nous allons nous familiariser avec les solveurs d'optimisation disponibles dans `scipy` et `cvxopt` à travers la résolution d'un problème jouet. 

In [ ]:
## Les imports de base
import numpy as np
import matplotlib.pyplot as plt

## Problème jouet 1 : Minimisation d'une fonction linéaire sous contrainte quadratique avec `minimize`

Considérons le problème de minimisation suivant (traité lors de la 2$^{ème}$ séance du cours) :

$$\begin{align}
\min_{\mathbf{x} \in \mathbb{R}^2} \quad & f(\mathbf{x}) = 2x_1 + x_2 \\
\text{tel que} \quad & g(\mathbf{x}) = 3x_1^2 + x_2^2 \leq 4
\end{align}
$$

Ce problème "faussement simple" est en réalité un problème QCQP (_Quadratically Constrained Quadratic Program_), donc avec une fonction objective quadratique et des contraintes d'inégalités quadratiques.
Ici la fonction objective est linéaire (donc, à fortiori, quadratique).

### Méthode graphique

Ce problème a été résolu graphiquement en cours :
1. nous avions commencé par visualiser l'espace admissible (lieu de sous niveau $4$ de la fonction $g$, qui est l'intérieur d'une ellipse)
2. nous avions ensuite tracé une courbe de niveau (celle de niveau $0$) de la fonction objective $f$ pour y positionner son vecteur normal et identifier le demi-espace positif et demi-espace négatif.
3. cherchant à **minimiser** la fonction objectif $f$, nous étions enfin parti à l'opposé du gradient $\nabla f$ en translatant la courbe de niveau jusqu'à arriver en bordure du lieu admissible.

Les 2 premières étapes sont représentées ci-dessous $\downarrow$

In [ ]:
# Définition de la grille
xx1 = np.linspace(-3, 3, 400)
xx2 = np.linspace(-3, 3, 400)
xx1, xx2 = np.meshgrid(xx1, xx2)

# Définition de la contrainte quadratique
constraint = 3*xx1**2 + xx2**2

# Définition de la fonction objective
objective = 2*xx1 + xx2

# Tracé de l'espace admissible
plt.figure(figsize=(7, 7))
plt.contourf(xx1, xx2, constraint, levels=[0, 4], colors=['gray', 'white'], alpha=0.5)
plt.contour(xx1, xx2, constraint, levels=[4], colors='black')
plt.text(1, 1.25, 'Lieu admissible', color='gray', fontsize=12, fontweight = 'bold')

# Tracé de la ligne de niveai 0 de la fonction objective
plt.contour(xx1, xx2, objective, levels=[0], colors='blue')
plt.text(-1, 2.25, r'Courbe de niveau 0 de $f$', color='blue', fontsize=12, fontweight = 'bold')

# Tracé du vecteur normal
plt.quiver(1, -2, 2, 1, angles='xy', scale_units='xy', scale=2, color='red')
plt.text(1.1, -1.4, r'vecteur normal $\nabla f$', color='red', fontsize=12, fontweight = 'bold')

# Labels et titre
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Espace admissible et courbe de niveau 0 de la fonction objective')
plt.grid(True)
plt.show()

In [ ]:
from scipy.optimize import minimize

### Résolution du problème jouet avec `scipy.optimize.minimize`

La fonction [`minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) du module `scipy.optimize` est un outil puissant pour résoudre des problèmes d'optimisation. Elle permet de minimiser une fonction objective en respectant des contraintes, que ce soit des contraintes d'égalité, des contraintes d'inégalité, ou des contraintes de bornes.
Elle se base pour cela sur divers algorithmes d'optimisation, et détermine automatiquement celui qui semble être le plus adapté en fonction des données fournies par l'utilisateur (identité et type des contraines, calcul de la Hessienne ou pas, etc).

Ses principaux arguments sont :

- **fun** : La fonction objective à minimiser. Cette fonction doit accepter un vecteur de variables et retourner un scalaire.
- **x0** : Valeurs initiales des variables, qui spécifie les points de départ pour l'algorithme d'optimisation. Peut être initialisé au vecteur nul.
- **method** : Méthode d'optimisation à utiliser (`'BFGS'` par défaut).
- **constraints** : Décrit les contraintes d'égalité et d'inégalité. Elles peuvent être spécifiées sous forme de dictionnaires ou de listes de dictionnaires.
- **bounds** : Bornes sur les variables. Peut être spécifié comme une séquence de tuples `(min, max)` pour chaque variable.
- **options** : Paramètres supplémentaires pour la méthode d'optimisation, tels que la tolérance et le nombre maximum d'itérations.

Pour plus d'informations, vous êtes renvoyés vers la [documentation officielle de `scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

#### Définition de la fonction objective via l'argument `fun`

L'argument `fun` de `scipy.optimize.minimize` représente la fonction objective à minimiser. Cette fonction doit être définie explicitement par l'utilisateur comme une fonction classique en Python, qui prend en entrée un vecteur `x` de variables et qui retourne une valeur scalaire.

Par exemple, si la fonction objective à minimiser est $f(\mathbf{x}) = x_1^2 + x_2^2 - 2 x_1 x_2 = (x_1 - x_2)^2$, elle s'implémentera dans `minimize` comme ceci :

```python
def objective_function(x):
    return x[0]**2 + x[1]**2 - 2*x[0]*x[1]


#### Définition des contraintes via l'argument `constraints`

L'argument `constraints` de `scipy.optimize.minimize` permet de spécifier les contraintes que les variables doivent satisfaire pendant l'optimisation. En particulier :
* ⚠️ les contraintes d'inégalité doivent être spécifiées de manière à ce que la fonction retourne une **valeur positive (ou nulle) lorsque la contrainte est respectée**.
* ⚠️ les contraintes d'égalité doivent être spécifiées de manière à ce que la fonction retourne une **valeur nulle lorsque la contrainte est respectée**.

Les contraintes sont définies sous forme d'un dictionnaire avec les clés suivantes :
- **`type`** : Indique le type de la contrainte : `'ineq'` pour les contraintes d'inégalité, et `'eq'` pour les contraintes d'égalité.
- **`fun`** : Une fonction qui prend un vecteur de variables et retourne la valeur de la contrainte. Sa définition est similaire à celle attendue pour la fonction objective.

Par exemple, si le problème d'optimisation doit respecter les deux contraintes suivantes $g(\mathbf{x}) = x_1^2 + x_2 \leq 3$ et $h(\mathbf{x}) = x_1 - 2 x_2 = 1$, elles pourront être définies comme une liste de dictionnaires pour `constraints` de la manère suivante :

```python
def ineq_constraint(x):
    return 3 - (x[0]**2 + x[1]) # doit être positive ou nulle si respectée

def eq_constraint(x):
    return x[0] - 2*x[1] - 1 # doit être nulle si respectée

constraints = [{'type': 'ineq', 'fun': ineq_constraint},
               {'type': 'ineq', 'fun': eq_constraint}]


#### Format du résultat de `minimize`

Le résultat de la fonction `scipy.optimize.minimize` est retourné sous la forme d'un objet `OptimizeResult` qui contient plusieurs informations importantes sur la solution trouvée. Ses principaux attributs sont :

- **`x`** : Un tableau numpy contenant les valeurs optimales des variables qui minimisent la fonction objective.
- **`fun`** : La valeur de la fonction objective à la solution optimale.
- **`success`** : Un booléen indiquant si l'optimisation a réussi (`True`) ou échoué (`False`).
- **`message`** : Un message décrivant le résultat de l'optimisation, indiquant la raison de l'arrêt de l'algorithme.
- **`status`** : Un code de statut indiquant le type de sortie de l'optimiseur (par exemple, succès ou échec).
- **`jac`** : Le gradient de la fonction objective à la solution optimale (disponible si le gradient est calculé).
- **`hess`** : La matrice Hessienne de la fonction objective à la solution optimale (disponible si la méthode utilisée le calcule).

Voici un exemple d'utilisation pour accéder à ces attributs après l'optimisation :

```python
# Résolution du problème
result = minimize(fun=objective_function, x0=x0, constraints=constraints)

# Afficher les résultats
print("Point optimal :", result.x)
print("Valeur optimale de la fonction objective :", result.fun)
print("Succès de l'optimisation :", result.success)
print("Message :", result.message)


### 🛠️ 🚧 👷  À vous de jouer !

Maintenant que vous avez compris (en théorie) comment définir la fonction objective et les contraintes, il est temps de mettre cela en pratique pour la résolution de notre problème QCQP :

$$\begin{align}
\min_{\mathbf{x} \in \mathbb{R}^2} \quad & f(\mathbf{x}) = 2x_1 + x_2 \\
\text{tel que} \quad & g(\mathbf{x}) = 3x_1^2 + x_2^2 \leq 4
\end{align}
$$

In [ ]:
# définition de la fonction objective
def objective_function(x):
    ??? # FIXME ⚠️

In [ ]:
# definition de la contrainte
def constraint_function(x):
    ??? # FIXME ⚠️

constraints = ??? # FIXME ⚠️

In [ ]:
# initialisation de la variable primale
x0 = ??? # FIXME ⚠️

In [ ]:
result = minimize(???) # FIXME ⚠️

# Affichage des résultats
print("Point optimal :", result.x)
print("Valeur optimale de la fonction objective :", result.fun)
print("Succès de l'optimisation :", result.success)
print("Message :", result.message)

Dans le cours, nous avions obtenu comme point optimal $\mathbf{x}^\star = \left(-\frac{4}{\sqrt{21}}, -\frac{6}{\sqrt{21}}\right)$ et comme valeur optimale $f^\star = -\frac{14}{\sqrt{21}}$

### 🛠️ 🚧 👷  À vous de jouer !

Vérifiez que les valeurs numériques obtenues concordent bien avec celles attendues (`np.isclose` peut s'avérer utile ici).

In [ ]:
??? # FIXME ⚠️ Alors, ça concorde ?

### Un beau dessin pour finir

On trace ici pour finir l'étape finale de la méthode de résolution graphique, une fois que la courbe de niveau a été translaté sur sa position optimale (donc tangente au lieu admissible), en y rajoutant le point optimal que vous avez trouvé grâce à `minimize`.

La cellule ci-dessous $\downarrow$ n'est donc à exécuter qu'une fois que vous avez résolu numériquement le problème d'optimisation QCQP 

In [ ]:
# Définition de la grille
xx1 = np.linspace(-3, 3, 400)
xx2 = np.linspace(-3, 3, 400)
xx1, xx2 = np.meshgrid(xx1, xx2)

# Définition de la contrainte quadratique
constraint = 3*xx1**2 + xx2**2

# Définition de la fonction objectuve
objective = 2*xx1 + xx2

# Tracé du lieu admissible
plt.figure(figsize=(7, 7))
plt.contourf(xx1, xx2, constraint, levels=[0, 4], colors=['gray', 'white'], alpha=0.5)
plt.contour(xx1, xx2, constraint, levels=[4], colors='black')
plt.text(1, 1.25, 'Lieu admissible', color='gray', fontsize=12, fontweight = 'bold')

# Tracé de plusieurs lignes de niveau de la fonction objective
plt.contour(xx1, xx2, objective, levels=np.linspace(-10, 10, 15), cmap='viridis', linestyles='dashed')

# Tracé de la ligne de niveau optimale
optimal_level = result.fun
plt.contour(xx1, xx2, objective, levels=[optimal_level], colors='red', linestyles='solid', linewidths=2)
plt.text(-0.3, -2.2, 'Courbe de niveau optimale', color='red', fontsize=12, fontweight = 'bold')

# Ajout du point optimal
x_optimal = result.x
plt.plot(x_optimal[0], x_optimal[1], 'ro', label=r'Point optimal $x^\star$')
plt.text(x_optimal[0]-0.05,x_optimal[1]-0.05, '(%1.2f, %1.2f)'%(x_optimal[0],x_optimal[1]),
         color='red', fontsize=12, fontweight = 'bold', ha='right')

# Labels et titre
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Courbe de niveau optimale et point optimal')
plt.grid(True)
plt.xlim([-2.5, 2.5])
plt.ylim([-2.5, 2.5])
plt.axhline(0, color='black',linewidth=0.5)
plt.axvline(0, color='black',linewidth=0.5)
plt.legend(loc='best',fontsize=14)
plt.show()

## Problème jouet 2 : Minimisation d'une fonction quadratique sous contrainte linéaire avec `cvxopt`

Dans l'exercice précédent, le problème d'optimisation à résoudre était de classe QCQP. Les programmes QCQP sont en effet des cas particuliers de [programmation semi-définie (SDP)](https://fr.wikipedia.org/wiki/Optimisation_SDP) ou de programmation en cône, et `cvxopt` peut les résoudre. Cependant, la résolution des QCQP implique des manipulations et des reformulations loin d'être triviales de prime abord pour adapter le problème à l'interface de `cvxopt`, et qui sont largement hors du périmètre de ce TP.

On se réfugie donc ici vers la résolution d'un problème QP (_quadratic programming_), où la fonction objective peut être quadratique, mais les contraintes restent toutes linéaires. Puisque c'est aussi la classe à laquelle appartient le SVM, cette simplification n'empêchera en rien la bonne tenue de la suite du TP (ouf).

### Nouveau problème QP

On considère ici le programme quadratique suivant : 

$$\begin{align}
\min_{\mathbf{x} \in \mathbb{R}^2} \quad & f(\mathbf{x}) = 3x_1^2 + x_2^2 \\
\text{tel que} \quad & g(\mathbf{x}) = 2x_1 + x_2 \geq 4
\end{align}
$$

La fonction objective est bien quadratique ✅, et la contrainte d'inégalité est bien linéaire ✅, c'est donc bien un problème QP.

### Visualisation du lieu admissible

Avant toute chose, commençons par visualiser le problème en traçant le lieu admissible et des courbes de niveau de la fonction objective. 

In [ ]:
# Définition de la grille
xx1 = np.linspace(-3, 3, 400)
xx2 = np.linspace(-3, 3, 400)
xx1, xx2 = np.meshgrid(xx1, xx2)

# Définition de la fonction objective
objective = 3*xx1**2 + xx2**2

# Définition de la contrainte linéaire
constraint = 2*xx1 + xx2

# Tracé du lieu admissible
plt.figure(figsize=(7, 7))
plt.contourf(xx1, xx2, constraint, levels=[4, np.inf], colors=['gray', 'white'], alpha=0.5)
plt.contour(xx1, xx2, constraint, levels=[4], colors='black')
plt.text(1, 2, 'Lieu admissible', color='black', fontsize=14, fontweight = 'bold')

# Tracé des lignes de niveau de la fonction objective
plt.contour(xx1, xx2, objective, levels=np.linspace(0, 30, 15), cmap='viridis', linestyles='dashed')

# Ajout de la légende
labels = ['Courbes de niveau de la fonction objective']
handles = [plt.Line2D([0], [0], linestyle='dashed', color='purple')]
plt.legend(handles, labels, loc='upper right', fontsize=12)
# Labels et titre
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Lieu admissible et courbes de niveau de la fonction objective')
plt.grid(True)
plt.show()

In [ ]:
import cvxopt
from cvxopt import printing
cvxopt.matrix_repr = printing.matrix_str_default
from cvxopt import matrix
from cvxopt.solvers import qp

### Résolution du problème jouet avec `cvxopt`

La bibliothèque `cvxopt` est un outil puissant pour résoudre des problèmes d'optimisation convexe en Python. Elle est particulièrement adaptée aux problèmes de programmation linéaire, quadratique, semi-définie ou en cône, offrant une interface flexible et efficace pour formuler et résoudre ces problèmes.

Pour plus de détails sur l'utilisation de `cvxopt`, les fonctions disponibles et des exemples d'application, vous êtes invités à consulter la [documentation officielle de `cvxopt`](https://cvxopt.org/documentation/).

#### Format des matrices dans `cvxopt`

`cvxopt` utilise un format spécifique de matrice pour représenter les paramètres des fonctions objectives et de contraintes, via sa propre classe [`matrix`](https://cvxopt.org/userguide/matrices.html). En première approximation, elle s'utilise comme un `np.array` pour l'instanciation :
```python
from cvxopt import matrix

# Création d'une matrice 2x2
A = matrix([[1.0, 2.0], [3.0, 4.0]])
```

La conversion d'un `np.array` en `matrix` se fait également très simplement :
```python
# Conversion d'un np.array en matrix
A = np.array([[1.0, 2.0], [3.0, 4.0]])
B = matrix(A)
```

Et la conversion en sens inverse également:
```python
# Conversion d'un np.array en matrix
A = matrix([[1.0, 2.0], [3.0, 4.0]])
B = np.array(A)
```
Bref, aucune raison de vous laisser déstabiliser par le format `matrix` de `cvxopt` 😉

#### Le solveur des problèmes QP de `cvxopt`

La fonction `qp` de `cvxopt.solvers` est utilisée pour résoudre les problèmes de programmation quadratique. Comme l'explique la [documentation](https://cvxopt.org/userguide/coneprog.html#cvxopt.solvers.qp), c'est en fait une interface vers une fonction plus générale (`coneqp`, qui permet elle de résoudre des problèmes plus généraux que QP).

La forme standard d'un problème QP pour le solveur `qp` est la suivante :

$$
\begin{aligned}
& \text{minimiser} & \frac{1}{2} \mathbf{x}^T \mathbf{P} \mathbf{x} + \mathbf{q}^T \mathbf{x} \\
& \text{sous les contraintes} & \mathbf{G}\mathbf{x} \leq \mathbf{h} \\
& & \mathbf{A}\mathbf{x} = \mathbf{b}
\end{aligned}
$$

où :
- $\mathbf{P}$ est une matrice symétrique définie positive.
- $\mathbf{q}$ est un vecteur.
- $\mathbf{G}$ et $\mathbf{h}$ définissent les contraintes d'inégalité.
- $\mathbf{A}$ et $\mathbf{b}$ définissent les contraintes d'égalité.

Ainsi, contrairement au solveur `minimize` de `scipy.optimize` qui permet (en théorie) de résoudre _n'importe_ quel problème d'optimisation via la manière dont sont définies les fonctions objectives et de contraintes, le solveur `qp` de `cvxopt.solvers` ne permet de résoudre que les problèmes d'optimisation qui s'écrivent sous la forme ci-dessus (c'est-à-dire les problèmes QP). Ceux-ci sont parfaitement déterminés par les définitions des matrices $\mathbf{P}$, $\mathbf{G}$, $\mathbf{A}$ et des vecteurs $\mathbf{q}, \mathbf{g}, \mathbf{b}$.

À noter que ces matrices et vecteurs doivent être définis via la classe `matrix` de `cvxopt`

#### Format du résultat de `cvxopt.solvers.qp`

Les différents solveurs de `cvxopt.solvers` (dont `qp`) retournent un résultat sous la forme d'un dictionnaire Python contenant plusieurs informations importantes sur la solution trouvée. Les principaux éléments de ce dictionnaire sont :

- **`x`** : Un vecteur (de type `matrix`) contenant la variable primale optimale. Par exemple, un vecteur de taille $(2 \times 1)$ pour un problème avec une variable primale $\mathbf{x} \in \mathbb{R}^2$.
- **`y`** : Un vecteur (de type `matrix`) contenant les variables duales (multiplicateurs de Lagrange) associées aux contraintes d'égalité (vide s'il n'y a pas de contraintes d'égalité).
- **`z`** : Un vecteur (de type `matrix`) contenant les variables duales (multiplicateurs de Lagrange) associées aux contraintes d'inégalité (vide s'il n'y a pas de contraintes d'inégalité).
- **`s`** : Un vecteur (de type `matrix`) contenant la valeur des variables de relâchement (_slack variables_) dans les contraintes d'inégalités (⚠️ ce n'est pas une variable duale en soit).
- **`status`** : Un indicateur du statut de l'optimisation. En ce qui nous concerne, on espère y voir `'optimal'`, ce qui indiquerait que le problème a bien été résolu et qu'une solution optimale a été trouvée par le solveur.
- **`gap`** : Le saut de dualité $p^\star - d^\star$, idéalement égal à 0.
- **`primal objective`** : La valeur optimale $p^\star$ de la fonction objective (primale).
- **`dual objective`** : La valeur optimale $d^\star$ de la fonction objective duale.
- **`iterations`** : Le nombre d'itérations effectuées par l'algorithme pour converger à la solution optimale.

Voici un exemple d'utilisation pour accéder à ces attributs après l'optimisation :

```python
# Résolution du problème
result = solvers.qp(P, q, G, h) # Si non mentionnées dans les arguments du solveur, les contraintes d'égalité A et b (ou d'inégalité G et h) sont implicitement définies comme étant égales à 0.

# Affichage des résultats
print("Point optimal :", result['x'])
print("Valeur optimale de la fonction objective :", result['primal objective'])
print("Variables duales des contraintes d'égalité :", result['y'])
print("Variables duales des contraintes d'inégalité :", result['z'])
print("Succès de l'optimisation :", result['status'])
print("Nombre d'itérations :", result['iterations'])


### 🛠️ 🚧 👷  À vous de jouer !

Utilisez le solveur `qp` pour résoudre numériquement le problème d'optimisation QP :

$$\begin{align}
\min_{\mathbf{x} \in \mathbb{R}^2} \quad & f(\mathbf{x}) = 3x_1^2 + x_2^2 \\
\text{tel que} \quad & g(\mathbf{x}) = 2x_1 + x_2 \geq 4
\end{align}
$$


⚠️ Il vous faut évidemment le mettre d'abord sous la forme standard attendue par `qp` $$
\begin{aligned}
& \text{minimiser} & \frac{1}{2} \mathbf{x}^T \mathbf{P} \mathbf{x} + \mathbf{q}^T \mathbf{x} \\
& \text{sous les contraintes} & \mathbf{G}\mathbf{x} \leq \mathbf{h} \\
& & \mathbf{A}\mathbf{x} = \mathbf{b}
\end{aligned}
$$
pour identifier les différentes matrices et vecteurs du problème à résoudre ici. ⚠️ Pensez à bien les définir au format `matrix`.

In [ ]:
# définition des matrices du problème QP
P = ??? # FIXME ⚠️
q = ??? # FIXME ⚠️
G = ??? # FIXME ⚠️
h = ??? # FIXME ⚠️
A = ??? # FIXME ⚠️
b = ??? # FIXME ⚠️

In [ ]:
result = ??? # FIXME ⚠️

# Affichage des résultats
print("Point optimal :", result['x'])
print("Valeur optimale de la fonction objective :", result['primal objective'])
print("Variables duales des contraintes d'égalité :", result['y'])
print("Variables duales des contraintes d'inégalité :", result['z'])
print("Succès de l'optimisation :", result['status'])
print("Nombre d'itérations :", result['iterations'])

### Un deuxième beau dessin

On complète ici la figure précédente de visualisation des courbes de niveau de la fonction objective et du lieu admissible avec la courbe de niveau optimale de la fonction objective et le point optimal, tous deux calculés par `cvxopt` dans les cellules précédentes.

In [ ]:
# Définition de la grille
xx1 = np.linspace(-2, 3, 400)
xx2 = np.linspace(-2, 3, 400)
xx1, xx2 = np.meshgrid(xx1, xx2)

# Définition de la fonction objective
objective = 3*xx1**2 + xx2**2

# Définition de la contrainte linéaire
constraint = 2*xx1 + xx2

# Tracé du lieu admissible
plt.figure(figsize=(7, 7))
plt.contourf(xx1, xx2, constraint, levels=[4, np.inf], colors=['gray', 'white'], alpha=0.5)
plt.contour(xx1, xx2, constraint, levels=[4], colors='black')
plt.text(1.25, 2.5, 'Lieu admissible', color='black', fontsize=14, fontweight = 'bold')

# Tracé des lignes de niveau de la fonction objective
plt.contour(xx1, xx2, objective, levels=np.linspace(0, 64, 24), cmap='viridis', linestyles='dashed')

# Récupération du point optimal et de la valeur optimale de cvxopt
optimal_point = np.array(result['x'])
optimal_value = np.array(result['primal objective'])

# Ajout de la ligne de niveau optimale
plt.contour(xx1, xx2, objective, levels=[optimal_value], colors='red', linewidths=3)
# Ajout du point optimal
plt.plot(optimal_point[0], optimal_point[1], 'ro', label='Point optimal',markersize=8)

# Ajout de la légende
labels = ['Courbes de niveau de la fonction objective',
          'Courbe de niveau optimal',
          'Point optimal']
handles = [plt.Line2D([0], [0], linestyle='dashed', color='purple'),
           plt.Line2D([0], [0], linestyle='solid', color='red'),
           plt.Line2D([0], [0], linestyle='none', marker='o', color='red')]
plt.legend(handles, labels, loc='lower right', fontsize=12)
# Labels et titre
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Lieu admissible et courbes de niveau de la fonction objective')
plt.grid(True)
plt.show()

## Bravo ! 👏🍻

Vous en avez terminé avec cet exercice de prise en main des solveurs de `scipy` et `cvxopt` ! Il est maintenant temps de s'attaquer à un problème un peu plus trapu : le [SVM à marge dure](TP_SVM_exo2.ipynb) !